In [1]:
!pip install transformers torch scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00


In [26]:
#Load and Explore the dataset
import json

# Load the JSON file
with open('intents.json') as f:
    data = json.load(f)

# See what the structure looks like
import pprint
pprint.pprint(list(data.items())[:2])

[('intents',
  [{'patterns': ['Hi',
                 'Hey',
                 'Is anyone there?',
                 'Hi there',
                 'Hello',
                 'Hey there',
                 'Howdy',
                 'Hola',
                 'Bonjour',
                 'Konnichiwa',
                 'Guten tag',
                 'Ola'],
    'responses': ['Hello there. Tell me how are you feeling today?',
                  'Hi there. What brings you here today?',
                  'Hi there. How are you feeling today?',
                  'Great to see you. How do you feel currently?',
                  "Hello there. Glad to see you're back. What's going on in "
                  'your world right now?'],
    'tag': 'greeting'},
   {'patterns': ['Good morning'],
    'responses': ["Good morning. I hope you had a good night's sleep. How are "
                  'you feeling today? '],
    'tag': 'morning'},
   {'patterns': ['Good afternoon'],
    'responses': ['Good afternoon. How i

In [27]:
#Convert it into datafram
import pandas as pd

df = pd.DataFrame(data)
df.head()

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'Hey', ..."
1,"{'tag': 'morning', 'patterns': ['Good morning'..."
2,"{'tag': 'afternoon', 'patterns': ['Good aftern..."
3,"{'tag': 'evening', 'patterns': ['Good evening'..."
4,"{'tag': 'night', 'patterns': ['Good night'], '..."


In [28]:
# Expand the 'intents' column
intents = df['intents'].apply(pd.Series)

# Create a new DataFrame with 'text' and 'label'
rows = []
for _, row in intents.iterrows():
    tag = row['tag']
    patterns = row['patterns']
    for pattern in patterns:
        rows.append({"text": pattern, "label": tag})

train_df = pd.DataFrame(rows)
train_df.head()

,text,label
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hi there,greeting
4,Hello,greeting


In [29]:
# STEP2 Tokenize with HuggingFace Transformers
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [32]:
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast

# Encode labels to integers
le = LabelEncoder()
train_df['label_encoded'] = le.fit_transform(train_df['label'])

# Tokenizer #uncased → the model doesn’t care about upper/lowercase.
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the text
encodings = tokenizer(list(train_df['text']), truncation=True, padding=True)
#truncation=True → cuts long sentences to fit the model’s max length (usually 512 tokens)
#padding=True → adds [PAD] tokens so all sequences have the same length
print(encodings.keys)

<bound method BatchEncoding.keys of {'input_ids': [[101, 7632, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 4931, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2003, 3087, 2045, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 7632, 2045, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 7592, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 4931, 2045, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2129, 5149, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 7570, 2721, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 14753, 23099, 2099, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 12849, 23500, 5428, 4213, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 9535, 2368, 6415, 102, 0, 0, 0, 0

In [33]:
import torch # required for building deep learning models

class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Return a single sample (dict)
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
dataset = IntentDataset(encodings, train_df['label_encoded'].tolist())

In [34]:
from transformers import DistilBertForSequenceClassification

# Get the number of unique labels
num_labels = len(le.classes_)

# Load the model with the correct number of output classes
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [36]:
!wandb login YOUR_API_KEY
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mariamsaadmrm (mariamsaadmrm-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,4.372900
20,4.329400
30,4.265500
40,4.109600
50,3.982000
60,3.945700
70,3.802000
80,3.765400
90,3.610100
100,3.570500


TrainOutput(global_step=116, training_loss=3.9183303405498635, metrics={'train_runtime': 478.8279, 'train_samples_per_second': 1.938, 'train_steps_per_second': 0.242, 'total_flos': 6251209635840.0, 'train_loss': 3.9183303405498635, 'epoch': 4.0})

In [37]:
#Make Predictions
def predict_intent(message):
    # Tokenize the new message
    encoding = tokenizer(message, truncation=True, padding=True, return_tensors='pt')

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits

    # Get the predicted class (highest logit)
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Decode the predicted class to its intent label
    predicted_intent = le.inverse_transform([predicted_class])[0]
    return predicted_intent

In [38]:
# Example messages
messages = [
    "Hello, how are you?",
    "I'm feeling down today.",
    "Good morning, everyone!",
    "I'm not sure what to do."
]

# Predict the intent for each message
for msg in messages:
    print(f"Message: {msg} | Predicted Intent: {predict_intent(msg)}")

Message: Hello, how are you? | Predicted Intent: about
Message: I'm feeling down today. | Predicted Intent: happy
Message: Good morning, everyone! | Predicted Intent: happy
Message: I'm not sure what to do. | Predicted Intent: happy


In [39]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(dataset):
    # Prepare the inputs and labels
    inputs = [tokenizer(text, truncation=True, padding=True, return_tensors='pt') for text in train_df['text']]
    labels = train_df['label_encoded'].tolist()

    # Make predictions for all inputs
    predictions = []
    for encoding in inputs:
        with torch.no_grad():
            outputs = model(**encoding)
            logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
        predictions.append(predicted_class)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy}")

    # Detailed classification report (precision, recall, F1-score)
    print("\nClassification Report:")
    print(classification_report(labels, predictions, target_names=le.classes_))

# Evaluate the model
evaluate_model(dataset)

Accuracy: 0.4482758620689655

Classification Report:
                     precision    recall  f1-score   support

              about       0.38      1.00      0.55         8
          afternoon       0.00      0.00      0.00         1
            anxious       0.00      0.00      0.00         2
                ask       0.00      0.00      0.00         1
             casual       0.23      0.45      0.30        11
           creation       0.00      0.00      0.00         3
              death       1.00      1.00      1.00         6
            default       0.73      1.00      0.84         8
          depressed       0.00      0.00      0.00         4
               done       1.00      0.60      0.75         5
            evening       0.00      0.00      0.00         1
             fact-1       0.00      0.00      0.00         2
            fact-10       0.00      0.00      0.00         1
            fact-11       0.00      0.00      0.00         1
            fact-12       0.00 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
print(num_labels)
print(train_df)

80
                                                  text     label  \
0                                                   Hi  greeting   
1                                                  Hey  greeting   
2                                     Is anyone there?  greeting   
3                                             Hi there  greeting   
4                                                Hello  greeting   
..                                                 ...       ...   
227                       How do I know if I'm unwell?   fact-29   
228  How can I maintain social connections? What if...   fact-30   
229  What's the difference between anxiety and stress?   fact-31   
230  What's the difference between sadness and depr...   fact-32   
231          difference between sadness and depression   fact-32   

     label_encoded  
0               44  
1               44  
2               44  
3               44  
4               44  
..             ...  
227             32  
228         

In [42]:
intent_responses = {
    "greeting": "Hey there! How can I help you today? 😊",
    "sadness": "I'm really sorry you're feeling this way. Want to talk about it?",
    "happy": "That's great to hear! Tell me more!",
    "morning": "Good morning! Hope you slept well!",
    "evening": "Good evening! How was your day?",
    "thanks": "You're very welcome! 🙏",
    "default": "I'm not sure I understand. Could you rephrase that?"
}

In [43]:
def chatbot_response(user_input):
    intent = predict_intent(user_input)  # Predict the intent from the message
    response = intent_responses.get(intent, intent_responses["default"])
    return response

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit']:
        print("Bot: Goodbye! 👋")
        break
    print("Bot:", chatbot_response(user_input))

You: hi
Bot: Hey there! How can I help you today? 😊
You: hello
Bot: Hey there! How can I help you today? 😊
You: wow, im feeling awsome
Bot: I'm not sure I understand. Could you rephrase that?
You: im feeling good
Bot: That's great to hear! Tell me more!
You: my code is working well
Bot: That's great to hear! Tell me more!
You: but my arm feel a little of pain
Bot: I'm not sure I understand. Could you rephrase that?
You: im sad
Bot: Hey there! How can I help you today? 😊
You: i feel pain
Bot: I'm not sure I understand. Could you rephrase that?
You: i have headache
Bot: That's great to hear! Tell me more!
You: its bad
Bot: Hey there! How can I help you today? 😊
You: im done
Bot: Hey there! How can I help you today? 😊
You: goodbye
Bot: I'm not sure I understand. Could you rephrase that?
You: good morning
Bot: I'm not sure I understand. Could you rephrase that?
You: goodmorning
Bot: Hey there! How can I help you today? 😊
You: morning
Bot: Hey there! How can I help you today? 😊
You: Good mo